In [1]:
# Importing packages

import pandas as pd
import numpy as np
import pybaseball
import warnings

In [2]:
warnings.filterwarnings('ignore')

Today, I am doing the simple task of computing RE24 and publishing my work online. This is by no means an original task, but I found it odd that there were no public RE24 tables available.

## Loading the Statcast Data

We will use Statcast data from 2023 to start. Then we will apply the same methodology to the past few years.

I will use try/except here to make sure we only need to use PyBaseball to load data once. After saving it, we can easily load it back and do not need to query the data from the API again.

In [3]:
pybaseball.cache.enable()

try:
    df = pd.read_csv('statcast2023.csv')
except FileNotFoundError:
    df = pybaseball.statcast(start_dt='2023-01-01', end_dt='2023-12-31')
    df.to_csv('statcast2023.csv')
    
try:
    df = df.drop('Unnamed: 0', axis = 1)
except:
    pass

df.head(10)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
0,CU,2023-11-01,84.9,-1.19,6.12,"Sborz, Josh",606466,622250,strikeout,called_strike,...,0,0,5,Infield shade,Standard,26.0,0.000,-0.090,NaN,NaN
1,FF,2023-11-01,96.6,-0.69,6.24,"Sborz, Josh",606466,622250,NaN,ball,...,0,0,5,Strategic,Standard,206.0,0.000,0.016,NaN,NaN
2,CU,2023-11-01,84.5,-1.27,6.11,"Sborz, Josh",606466,622250,NaN,called_strike,...,0,0,5,Standard,Standard,24.0,0.000,-0.027,NaN,NaN
3,FF,2023-11-01,95.4,-0.80,6.23,"Sborz, Josh",606466,622250,NaN,foul,...,0,0,5,Standard,Standard,204.0,0.000,-0.022,NaN,NaN
4,FF,2023-11-01,95.6,-0.83,6.19,"Sborz, Josh",606466,622250,NaN,ball,...,0,0,5,Standard,Standard,207.0,0.000,0.016,NaN,NaN
5,FF,2023-11-01,95.6,-0.66,6.25,"Sborz, Josh",682998,622250,field_out,hit_into_play,...,0,0,5,Standard,Standard,208.0,-0.002,-0.194,NaN,NaN
6,CU,2023-11-01,83.9,-1.20,6.10,"Sborz, Josh",682998,622250,NaN,ball,...,0,0,5,Standard,Standard,28.0,0.000,0.025,NaN,NaN
7,CU,2023-11-01,86.3,-1.02,6.17,"Sborz, Josh",672695,622250,strikeout,called_strike,...,0,0,5,Infield shade,Standard,24.0,-0.005,-0.173,NaN,NaN
8,FF,2023-11-01,96.3,-0.83,6.32,"Sborz, Josh",672695,622250,NaN,foul,...,0,0,5,Strategic,Standard,204.0,0.000,0.000,NaN,NaN
9,CU,2023-11-01,85.6,-1.19,6.17,"Sborz, Josh",672695,622250,NaN,ball,...,0,0,5,Standard,Standard,20.0,0.000,0.022,NaN,NaN


Before we get started with anything, we'll also want to just filter for regular season games. The dataset identifies other types of games, such as spring training and postseason games, in the `game_type` column. Although those samples tend to be smaller compared to regular season, we still want to just focus on the regular season for simplicity.

In [4]:
df = df[df['game_type'] == 'R']

## Identifying the Columns

In [5]:
# Viewing the columns in the dataset

df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

For this computation, I am roughly following the Run Expectancy table chapter from the *Analyzing Baseball Data with R* book by Jim Albert, Benjamin Baumer, and Max Marchi available online [here](https://beanumber.github.io/abdwr3e/05-runsexpectancy.html).

The authors start by creating a unique half-inning ID by combining the strings of the `game_id`, `inn_ct`, and `bat_home_id`. Our versions of that are `game_pk`, `inning`, and `inning_topbot`.

In [6]:
df['half_inning_id'] = df['game_pk'].astype(str) + df['inning'].astype(str) + df['inning_topbot'].astype(str)

We're also going to need the runs scored in each half inning and to note the runs before. Luckily, Statcast makes this easy for us because we have the columns `bat_score`, which gives the pre-pitch bat team score, and `post_bat_score`, which gives the post-pitch bat team score. You cannot score while pitching/on defense, so we can start by finding out how many runs were scored on any individual play, then group by half inning and sum those runs.

In [7]:
# It makes sense for us to check this calculation is correct
# by checking the counts for each run
# We can't have more than 4 runs scored on a play

df['runs_scored_on_play'] = df['post_bat_score'] - df['bat_score']
df['runs_scored_on_play'].value_counts()

runs_scored_on_play
0    700665
1     13192
2      3210
3       749
4       129
Name: count, dtype: int64

In addition to this, we'll also keep track of the runs at the start of the inning and at the end of the inning, which follows a similar process to the runs scored in an inning.

In [8]:
RunsOnPlay = df[['half_inning_id', 'bat_score', 'post_bat_score', 'runs_scored_on_play']]
RunsOnPlay = RunsOnPlay.rename(columns = {'bat_score': 'runs_before_inning', 'post_bat_score': 'runs_after_inning',
                                             'runs_scored_on_play': 'runs_scored_in_inning'})

RunsBeforeInning = RunsOnPlay[['half_inning_id', 'runs_before_inning']].groupby(['half_inning_id']).min()
RunsScoredDuringInning = RunsOnPlay[['half_inning_id', 'runs_scored_in_inning']].groupby(['half_inning_id']).sum()
RunsAfterInning = RunsOnPlay[['half_inning_id', 'runs_after_inning']].groupby(['half_inning_id']).max()

RunsInInning = RunsBeforeInning.join(RunsScoredDuringInning, how = 'left').join(RunsAfterInning, how = 'left')
RunsInInning

,runs_before_inning,runs_scored_in_inning,runs_after_inning
half_inning_id,,,
7163521Bot,0,0,0
7163521Top,0,0,0
7163522Bot,0,2,2
7163522Top,0,0,0
7163523Bot,2,0,2
...,...,...,...
7187827Top,8,2,10
7187828Bot,4,3,7
7187828Top,10,0,10


Jumping a bit ahead of the book, we are now going to identify and remove the half-innings where the home team walks off their opponent. Since we do not know how many runs would have ultimately been scored in an inning that ends before 3 outs are recorded (and we should also acknowledge that teams may play differently when they know exactly how many runs they will need to win), we do not want to include them in the run expectancy table.

We know that walk-offs are always the very last pitch of the game, so we will find every game's last pitch, then identify whether the batting team takes a lead on that pitch, and remove that half-inning.

Note: we would normally be able to simplify the task a bit by identifying such cases in the 9th inning or later, but a rule active during 2020-2021 allowed for 7-inning doubleheaders at one point in a full season. Hence, it could be possible to walk-off an opponent in the 7th inning (as opposed to the 9th) or walk-off an opponent in the 8th or 9th inning, which would be extra innings.

In [9]:
# We identify the last at-bat of every game, then the last pitch of each at-bat
# Use an inner join to identify the last pitch of every game

inning_startend = df[['game_pk', 'half_inning_id', 'inning_topbot', 'at_bat_number', 'pitch_number', 'events', 
               'bat_score', 'post_bat_score', 'post_away_score', 'post_home_score']]
inning_startend['ab_id'] = inning_startend['half_inning_id'].astype(str) + inning_startend['at_bat_number'].astype(str)
last_ab = inning_startend.loc[inning_startend.groupby(['game_pk'])['at_bat_number'].idxmax()]
last_pitch_of_ab = inning_startend.loc[inning_startend.groupby(['ab_id'])['pitch_number'].idxmax()]
last_pitch_of_game = pd.merge(last_ab, last_pitch_of_ab, how = 'inner')
last_pitch_of_game

,game_pk,half_inning_id,inning_topbot,at_bat_number,pitch_number,events,bat_score,post_bat_score,post_away_score,post_home_score,ab_id
0,716352,7163529Top,Top,70,3,strikeout,2,2,2,5,7163529Top70
1,716353,7163539Top,Top,70,5,strikeout,3,3,3,4,7163539Top70
2,716354,7163549Bot,Bot,92,3,field_out,9,9,10,9,7163549Bot92
3,716355,7163559Bot,Bot,72,3,field_out,1,1,9,1,7163559Bot72
4,716356,71635611Bot,Bot,88,4,strikeout,1,1,2,1,71635611Bot88
...,...,...,...,...,...,...,...,...,...,...,...
2425,718778,7187789Bot,Bot,81,1,field_out,2,2,7,2,7187789Bot81
2426,718779,7187799Top,Top,79,6,strikeout,7,7,7,11,7187799Top79
2427,718780,7187809Bot,Bot,84,5,field_out,2,2,7,2,7187809Bot84
2428,718781,7187819Top,Top,67,4,strikeout,0,0,0,5,7187819Top67


In [10]:
# Walkoffs are when the home team takes the lead on the last pitch of the game
# And we filter for bottom of the inning because the home team can only score in the bottom

confirmed_walkoffs = last_pitch_of_game[(last_pitch_of_game['post_away_score'] < last_pitch_of_game['post_home_score']) & 
                                        (last_pitch_of_game['inning_topbot'] == 'Bot')]
confirmed_walkoffs

,game_pk,half_inning_id,inning_topbot,at_bat_number,pitch_number,events,bat_score,post_bat_score,post_away_score,post_home_score,ab_id
7,716359,71635911Bot,Bot,85,1,stolen_base_3b,2,3,2,3,71635911Bot85
33,716385,71638510Bot,Bot,78,3,double,3,4,3,4,71638510Bot78
45,716397,7163979Bot,Bot,68,3,double,1,3,2,3,7163979Bot68
52,716404,7164048Bot,Bot,62,3,strikeout,1,1,0,1,7164048Bot62
60,716412,71641210Bot,Bot,79,2,single,5,6,5,6,71641210Bot79
...,...,...,...,...,...,...,...,...,...,...,...
2331,718684,71868410Bot,Bot,79,3,single,2,3,2,3,71868410Bot79
2345,718698,7186989Bot,Bot,75,7,home_run,6,7,6,7,7186989Bot75
2354,718707,7187079Bot,Bot,73,1,single,3,4,3,4,7187079Bot73
2369,718722,7187229Bot,Bot,71,5,home_run,4,5,4,5,7187229Bot71


In [11]:
# Removing all walkoff half innings from our half innings dataframe

RunsInInning = RunsInInning.reset_index()
RunsInInning = RunsInInning[~RunsInInning['half_inning_id'].isin(confirmed_walkoffs['half_inning_id'])]
RunsInInning

,half_inning_id,runs_before_inning,runs_scored_in_inning,runs_after_inning
0,7163521Bot,0,0,0
1,7163521Top,0,0,0
2,7163522Bot,0,2,2
3,7163522Top,0,0,0
4,7163523Bot,2,0,2
...,...,...,...,...
43200,7187827Top,8,2,10
43201,7187828Bot,4,3,7
43202,7187828Top,10,0,10
43203,7187829Bot,7,2,9


And now, as the book instructs, we will join this dataframe to the overall Statcast dataframe with the newly created columns that the run expectancy table will care about. We'll also create a new column in this dataframe for the runs scored in the rest of the inning.

In [12]:
df = df.set_index(['half_inning_id']).join(RunsInInning.set_index(['half_inning_id']), how = 'left')
df['runs_rest_of_inning'] = df['runs_after_inning'] - df['bat_score']
df

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,runs_scored_on_play,runs_before_inning,runs_scored_in_inning,runs_after_inning,runs_rest_of_inning
half_inning_id,,,,,,,,,,,,,,,,,,,,,
7163679Bot,CH,2023-10-01,89.0,-2.80,5.59,"Robertson, Nick",677008,687798,field_out,hit_into_play,...,250.0,0.0,-0.090,NaN,NaN,0,1.0,0.0,1.0,0.0
7163679Bot,FF,2023-10-01,96.9,-2.40,5.90,"Robertson, Nick",677008,687798,NaN,foul,...,211.0,0.0,0.000,NaN,NaN,0,1.0,0.0,1.0,0.0
7163679Bot,CH,2023-10-01,90.0,-2.93,5.56,"Robertson, Nick",677008,687798,NaN,ball,...,250.0,0.0,0.016,NaN,NaN,0,1.0,0.0,1.0,0.0
7163679Bot,ST,2023-10-01,82.2,-3.09,5.55,"Robertson, Nick",677008,687798,NaN,ball,...,58.0,0.0,0.009,NaN,NaN,0,1.0,0.0,1.0,0.0
7163679Bot,CH,2023-10-01,89.2,-2.87,5.58,"Robertson, Nick",677008,687798,NaN,swinging_strike,...,257.0,0.0,-0.023,NaN,NaN,0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7187671Top,SI,2023-03-30,95.3,-3.11,5.24,"Castillo, Luis",680757,622491,NaN,called_strike,...,234.0,0.0,-0.079,NaN,NaN,0,0.0,0.0,0.0,0.0
7187671Top,FF,2023-03-30,94.2,-3.16,5.29,"Castillo, Luis",680757,622491,NaN,called_strike,...,229.0,0.0,-0.078,NaN,NaN,0,0.0,0.0,0.0,0.0
7187671Top,FF,2023-03-30,94.8,-3.12,5.30,"Castillo, Luis",680757,622491,NaN,ball,...,233.0,0.0,0.135,NaN,NaN,0,0.0,0.0,0.0,0.0


## Runner-Out States

The RE24 table lists run expectancy based on the occupied bases and outs combination in the game. We will need to simplify some of the Statcast data so that we can aggregate these combinations and report their run expectancy in the table.

In [13]:
df['1B_binary'] = np.where(df['on_1b'].notna(), 1, 0)
df['2B_binary'] = np.where(df['on_2b'].notna(), 1, 0)
df['3B_binary'] = np.where(df['on_3b'].notna(), 1, 0)
df['state'] = ("Bases: " + df['1B_binary'].astype(str) + df['2B_binary'].astype(str) + 
               df['3B_binary'].astype(str) + ", Outs: " + df['outs_when_up'].astype(str))

We will also want to both recreate the `ab_id` column and create a `pitch_id` column in case a bases and outs combination changes within an at-bat, as we'll see later.

In [14]:
df = df.reset_index()
df['ab_id'] = df['half_inning_id'].astype(str) + df['at_bat_number'].astype(str)
df['pitch_id'] = df['half_inning_id'].astype(str) + df['at_bat_number'].astype(str) + df['pitch_number'].astype(str)

We have now created all of the columns that run expectancy cares about, so we'll create a new dataframe to avoid showing all of the columns we won't be referring to.

In [15]:
RunsExpectancyData = df[['half_inning_id', 'ab_id', 'pitch_id', 'pitch_number', '1B_binary', '2B_binary', '3B_binary', 'outs_when_up',  'des', 
                         'runs_scored_on_play', 'runs_before_inning', 'runs_scored_in_inning',
                         'runs_after_inning', 'runs_rest_of_inning', 'state']]
RunsExpectancyData

,half_inning_id,ab_id,pitch_id,pitch_number,1B_binary,2B_binary,3B_binary,outs_when_up,des,runs_scored_on_play,runs_before_inning,runs_scored_in_inning,runs_after_inning,runs_rest_of_inning,state
0,7163679Bot,7163679Bot73,7163679Bot736,6,0,0,0,2,"Heston Kjerstad grounds out, first baseman Bob...",0,1.0,0.0,1.0,0.0,"Bases: 000, Outs: 2"
1,7163679Bot,7163679Bot73,7163679Bot735,5,0,0,0,2,"Heston Kjerstad grounds out, first baseman Bob...",0,1.0,0.0,1.0,0.0,"Bases: 000, Outs: 2"
2,7163679Bot,7163679Bot73,7163679Bot734,4,0,0,0,2,"Heston Kjerstad grounds out, first baseman Bob...",0,1.0,0.0,1.0,0.0,"Bases: 000, Outs: 2"
3,7163679Bot,7163679Bot73,7163679Bot733,3,0,0,0,2,"Heston Kjerstad grounds out, first baseman Bob...",0,1.0,0.0,1.0,0.0,"Bases: 000, Outs: 2"
4,7163679Bot,7163679Bot73,7163679Bot732,2,0,0,0,2,"Heston Kjerstad grounds out, first baseman Bob...",0,1.0,0.0,1.0,0.0,"Bases: 000, Outs: 2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717940,7187671Top,7187671Top1,7187671Top15,5,0,0,0,0,Steven Kwan pops out to third baseman Eugenio ...,0,0.0,0.0,0.0,0.0,"Bases: 000, Outs: 0"
717941,7187671Top,7187671Top1,7187671Top14,4,0,0,0,0,Steven Kwan pops out to third baseman Eugenio ...,0,0.0,0.0,0.0,0.0,"Bases: 000, Outs: 0"
717942,7187671Top,7187671Top1,7187671Top13,3,0,0,0,0,Steven Kwan pops out to third baseman Eugenio ...,0,0.0,0.0,0.0,0.0,"Bases: 000, Outs: 0"
717943,7187671Top,7187671Top1,7187671Top12,2,0,0,0,0,Steven Kwan pops out to third baseman Eugenio ...,0,0.0,0.0,0.0,0.0,"Bases: 000, Outs: 0"


We'll now ignore some of those columns in order to subset the data for only plays that change the bases and outs combination and/or score runs.

First we sort the table so the last pitches are at the top, then we can drop the duplicates that do not show any change in  bases occupied or outs recorded.

Note: we are actually just counting each of the at-bats once rather than actually checking for changes. This may overlook certain scenarios but we will make the assumption that this will not significantly affect the run expectancy table. We can also check if it is significantly affected when we validate this method with the 2016 data.

In [16]:
abs_with_changes = RunsExpectancyData[['ab_id', 'pitch_id', 'pitch_number', 'des', 'state', 'runs_scored_on_play']]
abs_with_changes = abs_with_changes.sort_values(['pitch_id', 'state', 'runs_scored_on_play'], ascending = False)
abs_with_changes = abs_with_changes.drop_duplicates(subset = ['ab_id', 'state'], keep = 'first')
abs_with_changes

,ab_id,pitch_id,pitch_number,des,state,runs_scored_on_play
713444,7187829Top84,7187829Top843,3,Ryan Mountcastle lines out sharply to right fi...,"Bases: 110, Outs: 2",0
713447,7187829Top83,7187829Top832,2,Anthony Santander singles on a sharp line driv...,"Bases: 100, Outs: 2",0
713449,7187829Top82,7187829Top822,2,Adley Rutschman singles on a ground ball to th...,"Bases: 000, Outs: 2",0
713451,7187829Top81,7187829Top814,4,"Cedric Mullins lines into a double play, secon...","Bases: 100, Outs: 0",0
713455,7187829Top80,7187829Top805,5,Jorge Mateo walks.,"Bases: 000, Outs: 0",0
...,...,...,...,...,...,...
4380,7163521Top2,7163521Top23,3,Anthony Volpe singles on a sharp line drive to...,"Bases: 100, Outs: 0",0
4384,7163521Top1,7163521Top19,9,DJ LeMahieu singles on a sharp line drive to c...,"Bases: 000, Outs: 0",0
4362,7163521Bot8,7163521Bot84,4,Salvador Perez strikes out swinging.,"Bases: 000, Outs: 2",0
4366,7163521Bot7,7163521Bot75,5,"Bobby Witt Jr. grounds out, second baseman Os...","Bases: 000, Outs: 1",0


In [17]:
RE24_data_2023 = RunsExpectancyData[RunsExpectancyData['pitch_id'].isin(abs_with_changes['pitch_id'])][['outs_when_up', '1B_binary', '2B_binary', '3B_binary', 'runs_rest_of_inning']]
RE24_data_2023['Bases'] = RE24_data_2023['1B_binary'].astype(str) + RE24_data_2023['2B_binary'].astype(str) + RE24_data_2023['3B_binary'].astype(str)
RE24_data_2023 = RE24_data_2023.drop(columns = ['1B_binary', '2B_binary', '3B_binary'])
RE24_data_2023 = RE24_data_2023.rename(columns = {'outs_when_up': 'Outs', 'runs_rest_of_inning': 'Runs'})
RE24_data_2023 = RE24_data_2023.groupby(['Bases', 'Outs']).mean()
RE24_data_2023 = RE24_data_2023.reset_index()
RE24_data_2023

,Bases,Outs,Runs
0,000,0,0.513162
1,000,1,0.272968
2,000,2,0.104097
3,001,0,1.464444
4,001,1,0.972603
5,001,2,0.357815
6,010,0,1.131379
7,010,1,0.705946
8,010,2,0.308621
9,011,0,1.916788


Lastly, we want to reshape the table into a more presentable format:

In [18]:
RE24_2023 = RE24_data_2023.pivot(index='Bases', columns='Outs', values='Runs')
RE24_2023

Outs,0,1,2
Bases,,,
000,0.513162,0.272968,0.104097
001,1.464444,0.972603,0.357815
010,1.131379,0.705946,0.308621
011,1.916788,1.364883,0.526152
100,0.900556,0.537914,0.239489
101,1.825994,1.177682,0.526352
110,1.458616,0.945819,0.471745
111,2.251381,1.589162,0.771806


And here is the final function:

In [19]:
def compute_RE24(startyear = 2023, endyear = 2024):
    for year in range(startyear, endyear + 1):
        try:
            df_year = pd.read_csv('statcast{}.csv'.format(year))
        except FileNotFoundError:
            df_year = pybaseball.statcast(start_dt='{}-01-01'.format(year), end_dt='{}-12-31'.format(year))
            df_year.to_csv('statcast{}.csv'.format(year))
        if 'df_multiyear' in locals() or 'df_multiyear' in globals():
            df_multiyear = df_multiyear.append(df_year, ignore_index=True)
        else:
            df_multiyear = df_year
    
    df = df_multiyear
    
    try:
        df = df.drop('Unnamed: 0', axis = 1)
    except:
        pass
            
    df = df[df['game_type'] == 'R']
    
    df['half_inning_id'] = df['game_pk'].astype(str) + df['inning'].astype(str) + df['inning_topbot'].astype(str)
    
    df['runs_scored_on_play'] = df['post_bat_score'] - df['bat_score']
    
    RunsOnPlay = df[['half_inning_id', 'bat_score', 'post_bat_score', 'runs_scored_on_play']]
    RunsOnPlay = RunsOnPlay.rename(columns = {'bat_score': 'runs_before_inning', 'post_bat_score': 'runs_after_inning',
                                                 'runs_scored_on_play': 'runs_scored_in_inning'})

    RunsBeforeInning = RunsOnPlay[['half_inning_id', 'runs_before_inning']].groupby(['half_inning_id']).min()
    RunsScoredDuringInning = RunsOnPlay[['half_inning_id', 'runs_scored_in_inning']].groupby(['half_inning_id']).sum()
    RunsAfterInning = RunsOnPlay[['half_inning_id', 'runs_after_inning']].groupby(['half_inning_id']).max()

    RunsInInning = RunsBeforeInning.join(RunsScoredDuringInning, how = 'left').join(RunsAfterInning, how = 'left')
    
    inning_startend = df[['game_pk', 'half_inning_id', 'inning_topbot', 'at_bat_number', 'pitch_number', 'events',
                          'bat_score', 'post_bat_score', 'post_away_score', 'post_home_score']]
    inning_startend['ab_id'] = inning_startend['half_inning_id'].astype(str) + inning_startend['at_bat_number'].astype(str)
    last_ab = inning_startend.loc[inning_startend.groupby(['game_pk'])['at_bat_number'].idxmax()]
    last_pitch_of_ab = inning_startend.loc[inning_startend.groupby(['ab_id'])['pitch_number'].idxmax()]
    last_pitch_of_game = pd.merge(last_ab, last_pitch_of_ab, how = 'inner')
    
    confirmed_walkoffs = last_pitch_of_game[(last_pitch_of_game['post_away_score'] < last_pitch_of_game['post_home_score']) & 
                                        (last_pitch_of_game['inning_topbot'] == 'Bot')]
    
    RunsInInning = RunsInInning.reset_index()
    RunsInInning = RunsInInning[~RunsInInning['half_inning_id'].isin(confirmed_walkoffs['half_inning_id'])]
    
    df = df.set_index(['half_inning_id']).join(RunsInInning.set_index(['half_inning_id']), how = 'left')
    df['runs_rest_of_inning'] = df['runs_after_inning'] - df['bat_score']
    
    df['1B_binary'] = np.where(df['on_1b'].notna(), 1, 0)
    df['2B_binary'] = np.where(df['on_2b'].notna(), 1, 0)
    df['3B_binary'] = np.where(df['on_3b'].notna(), 1, 0)
    df['state'] = ("Bases: " + df['1B_binary'].astype(str) + df['2B_binary'].astype(str) + 
                   df['3B_binary'].astype(str) + ", Outs: " + df['outs_when_up'].astype(str))
    
    df = df.reset_index()
    df['ab_id'] = df['half_inning_id'].astype(str) + df['at_bat_number'].astype(str)
    df['pitch_id'] = df['half_inning_id'].astype(str) + df['at_bat_number'].astype(str) + df['pitch_number'].astype(str)
    
    RunsExpectancyData = df[['half_inning_id', 'ab_id', 'pitch_id', 'pitch_number', '1B_binary', '2B_binary', '3B_binary', 'outs_when_up',  'des', 
                         'runs_scored_on_play', 'runs_before_inning', 'runs_scored_in_inning',
                         'runs_after_inning', 'runs_rest_of_inning', 'state']]
    
    abs_with_changes = RunsExpectancyData[['ab_id', 'pitch_id', 'pitch_number', 'des', 'state', 'runs_scored_on_play']]
    abs_with_changes = abs_with_changes.sort_values(['pitch_id', 'state', 'runs_scored_on_play'], ascending = False)
    abs_with_changes = abs_with_changes.drop_duplicates(subset = ['ab_id', 'state'], keep = 'first')
    
    RE24_data_year = RunsExpectancyData[RunsExpectancyData['pitch_id'].isin(abs_with_changes['pitch_id'])][['outs_when_up', '1B_binary', '2B_binary', '3B_binary', 'runs_rest_of_inning']]
    RE24_data_year['Bases'] = RE24_data_year['1B_binary'].astype(str) + RE24_data_year['2B_binary'].astype(str) + RE24_data_year['3B_binary'].astype(str)
    RE24_data_year = RE24_data_year.drop(columns = ['1B_binary', '2B_binary', '3B_binary'])
    RE24_data_year = RE24_data_year.rename(columns = {'outs_when_up': 'Outs', 'runs_rest_of_inning': 'Runs'})
    RE24_data_year = RE24_data_year.groupby(['Bases', 'Outs']).mean()
    RE24_data_year = RE24_data_year.reset_index()
    
    RE24_year = RE24_data_year.pivot(index='Bases', columns='Outs', values='Runs')
    
    return RE24_year


We now input the year 2016 to compare what this code generates to the RE24 table from the book:

In [20]:
# RE24 Table from my code

compute_RE24(startyear = 2016, endyear = 2016)

Outs,0,1,2
Bases,,,
000,0.498069,0.267858,0.106140
001,1.346979,0.936888,0.371287
010,1.133786,0.672314,0.312365
011,1.928788,1.357373,0.547457
100,0.858181,0.511728,0.220643
101,1.722513,1.196618,0.478230
110,1.444970,0.919658,0.413916
111,2.106105,1.537215,0.695272


| Bases | 0 out | 1 outs | 2 outs |
| --- | --- | --- | --- |
| **000** | 0.498 | 0.268 | 0.106 |
| **001** | 1.35 | 0.937 | 0.372 |
| **010** | 1.13 | 0.673 | 0.312 |
| **011** | 1.93 | 1.36 | 0.548 | 
| **100** | 0.858 | 0.512 | 0.220 |
| **101** | 1.72 | 1.20 | 0.478 |
| **110** | 1.44 | 0.921 | 0.414 |
| **111** | 2.11 | 1.54 | 0.695 |

The result is similar enough that we would have fairly confidence in our results for 2023 and our methodology. We could potentially attribute those differences to the way we are counting certain types of at-bats, but the difference is marginal and would not significantly impact any findings we would be interested in involving a RE24 table.